In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
%load_ext gams.magic
m = gams.exchange_container

In [3]:
gams.reset()
m = gams.exchange_container
df = pd.read_csv('food_nutrients.csv', index_col="Item")
df 

,Description,Brand,Calories,Serving Size,Nutrients
Item,,,,,
chicken,CHICKEN,Keystone Brand Meats Inc,107.0 kcal,56.0 g,"Fatty acids, total trans: 0.0 g, Carbohydrate,..."
yogurt,YOGURT,Agri-Service LLC,94.0 kcal,170.0 ml,"Iron, Fe: 0.0 mg, Calcium, Ca: 118.0 mg, Fiber..."
cereal,CEREAL,"General Mills, Inc.",387.0 kcal,31.0 g,"Fatty acids, total polyunsaturated: 1.61 g, Ir..."
Corn,CORN,"Kcb Industries, L.L.C.",545.0 kcal,33.0 g,"Vitamin D (D2 + D3), International Units: 0.0 ..."
Potatoes,POTATOES,THE LITTLE POTATO COMPANY,74.0 kcal,148.0 g,"Iron, Fe: 0.73 mg, Cholesterol: 0.0 mg, Calciu..."
...,...,...,...,...,...
Ice cream,ICE CREAM,"Clover Stornetta Farms, Inc.",205.0 kcal,88.0 g,"Energy: 205.0 kcal, Carbohydrate, by differenc..."
cookies,COOKIES,KNO,441.0 kcal,100.0 g,"Fiber, total dietary: 0.0 g, Sodium, Na: 0.0 m..."
brownies,BROWNIES,TWO-BITE,460.0 kcal,50.0 g,"Iron, Fe: 2.88 mg, Fiber, total dietary: 0.0 g..."


In [4]:
df['Calories'] = df['Calories'].str.extract('(\d+\.\d+)').astype(float)

In [5]:
df['Serving Size'] = df['Serving Size'].str.extract('(\d+\.\d+)').astype(float)

In [6]:
def extract_nutrients(row):
    nutrients = {}
    items = re.split(', ', row['Nutrients'])
    for item in items:
        match = re.match(r'(.+?): (.+?) (.+)', item)
        if match:
            nutrient, value = match.groups()[:2]
            nutrients[nutrient] = float(value) if '.' in value else int(value)
    return pd.Series(nutrients)

# Apply the function to each row and concatenate the result with the original DataFrame
df = pd.concat([df, df.apply(extract_nutrients, axis=1)], axis=1)

# Drop the original 'Nutrients' column
df = df.drop('Nutrients', axis=1)

In [7]:
df.head()

,Description,Brand,Calories,Serving Size,Alanine,Arginine,Ash,Aspartic acid,Ca,Caffeine,...,other,soluble,total,total ascorbic acid,total dietary,total including NLEA,total monounsaturated,total polyunsaturated,total saturated,total trans
Item,,,,,,,,,,,,,,,,,,,,,
chicken,CHICKEN,Keystone Brand Meats Inc,107.0,56.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,0.0,0.0,0.00,NaN,NaN,0.89,0.0
yogurt,YOGURT,Agri-Service LLC,94.0,170.0,NaN,NaN,NaN,NaN,118.0,NaN,...,NaN,NaN,NaN,0.7,0.0,15.88,NaN,NaN,1.76,NaN
cereal,CEREAL,"General Mills, Inc.",387.0,31.0,NaN,NaN,NaN,NaN,323.0,NaN,...,48.0,NaN,NaN,19.4,6.5,29.03,1.61,1.61,0.00,0.0
Corn,CORN,"Kcb Industries, L.L.C.",545.0,33.0,NaN,NaN,NaN,NaN,61.0,NaN,...,NaN,NaN,NaN,NaN,6.1,24.24,NaN,NaN,4.55,0.0
Potatoes,POTATOES,THE LITTLE POTATO COMPANY,74.0,148.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,2.4,2.0,0.68,NaN,NaN,0.00,0.0


In [8]:
df.rename(columns={'by difference': 'Carbs'}, inplace=True)
df.rename(columns={'Total lipid (fat)': 'Fat'}, inplace=True)
df.rename(columns={'Ca': 'Calcium'}, inplace=True)
df.rename(columns={'Vitamin D (D2 + D3)': 'Vitamin D'}, inplace=True)
df.rename(columns={'Serving Size': 'Serving'}, inplace=True)

In [9]:
df = df[["Calories", "Serving", 'Protein', "Carbs", "Fat", "Calcium"]]


In [10]:
np.random.seed(42)  # Setting a seed for reproducibility
cost_values = np.random.normal(5, 1, len(df))

df['Price'] = cost_values

In [11]:
df_limited = df.head(10)

In [12]:
df.fillna(0, inplace=True)

In [13]:
df = df.drop(['Rice Krispies', 'chili', 'Cornbread', "Instant oatmeal", 'Cheese curds', 'Peanut butter and jelly sandwiches'])
# df = df.drop(['chili'])
# df = df.drop(['Cornbread', "Instant oatmeal"])
# df = df.drop(['Peanut butter and jelly sandwiches'])

In [14]:
foods = df.index.values.tolist()
foods

['chicken',
 'yogurt',
 'cereal',
 'Corn',
 'Potatoes',
 'Green beans',
 'Carrots',
 'Tomatoes',
 'Bell peppers',
 'Onions',
 'Lettuce',
 'Spinach',
 'Broccoli',
 'Cauliflower',
 'Cucumbers',
 'Zucchini',
 'Peas',
 'Brussels sprouts',
 'Apples',
 'Bananas',
 'Grapes',
 'Oranges',
 'Strawberries',
 'Blueberries',
 'Watermelon',
 'Pears',
 'Cherries',
 'Plums',
 'Cantaloupe',
 'Raspberries',
 'Blackberries',
 'Kiwi',
 'Popcorn',
 'Pretzels',
 'Trail mix',
 'Beef jerky',
 'Potato chips',
 'Granola bars',
 'Applesauce',
 'peanuts',
 'almonds',
 'Crackers',
 'Cheese slices',
 'Fruit cups',
 'marshmallows',
 'chocolate',
 'graham crackers',
 'Milk',
 'Cheese',
 'steaks',
 'burgers',
 'Pork (bacon, ham)',
 'Eggs',
 'Wheat bread',
 'Oatmeal',
 'Pancake mix',
 'Pasta',
 'Rice',
 'Peaches',
 'Instant noodles',
 'Frozen pizza',
 'mac and cheese',
 'Canned soups',
 'cream cheese',
 'Chips and salsa',
 'Fruit snacks',
 'dried fruits',
 'String cheese',
 'Protein bar',
 'Coffee',
 'tea',
 'Energy dr

In [15]:
meats = ['chicken', 'steaks', 'burgers', 'Pork (bacon, ham)']
fruits = ['Apples','Bananas','Grapes','Oranges','Strawberries','Blueberries','Watermelon','Pears','Cherries','Plums','Cantaloupe','Raspberries','Blackberries','Kiwi','Peaches']  
vegetables = ['Corn','Potatoes','Green beans','Carrots','Tomatoes','Bell peppers','Onions','Lettuce','Spinach','Broccoli','Cauliflower','Cucumbers','Zucchini','Peas','Brussels sprouts','Salads']        
breakfast = ['yogurt','cereal', 'Eggs', 'Oatmeal','Pancake mix', 'cream cheese','Toaster pastries','Frozen waffles']  
frozen = ['Frozen pizza', 'Pizza rolls','mozzarella sticks','frozen vegetables']
drinks = ['Coffee','tea','Energy drinks','sports drinks','Fruit juices','smoothies','Soda']
desserts = ['Ice cream','cookies','brownies', 'marshmallows','chocolate'] 
snacks = ['Popcorn','Pretzels','Trail mix','Beef jerky','Potato chips','Granola bars','Applesauce','peanuts','almonds','Crackers','Cheese slices','Fruit cups','graham crackers', "Hummus",'Chips and salsa','Fruit snacks','dried fruits',"String cheese",'Protein bar']
dry = ["pita bread", 'Instant noodles', "mac and cheese", 'Pasta','Rice', 'Wheat bread','Canned soups']   
dairy = ['Cheese slices', 'Milk','Cheese',]
    

In [16]:
len(meats) + len(fruits) + len(vegetables) + len(breakfast) + len(frozen) + len(drinks) + len(desserts) + len(snacks) + len(dry)

85

In [17]:
len(df)


87

In [18]:
leftovers = []
for i in foods:
    if i in meats:
        continue
    if i in fruits:
        continue
    if i in vegetables:
        continue
    if i in breakfast:
        continue
    if i in frozen:
        continue
    if i in drinks:
        continue
    if i in desserts:
        continue
    if i in snacks:
        continue
    if i in dry:
        continue
    if i in dairy:
        continue
    
leftovers

[]

In [19]:
import pandas as pd

# Assuming df is your original DataFrame
# Assuming meats, fruits, vegetables, etc. are lists containing food items of each type

# Create an empty dictionary to store DataFrames for each type of food
food_dict = {}

# Iterate through the food items in the original DataFrame
for i in df.index.values:
    if i in meats:
        if 'Meats' not in food_dict:
            food_dict['Meats'] = pd.DataFrame(columns=df.columns)
        food_dict['Meats'] = pd.concat([food_dict['Meats'], df.loc[[i]]])
    elif i in fruits:
        if 'Fruits' not in food_dict:
            food_dict['Fruits'] = pd.DataFrame(columns=df.columns)
        food_dict['Fruits'] = pd.concat([food_dict['Fruits'], df.loc[[i]]])
    elif i in vegetables:
        if 'Vegetables' not in food_dict:
            food_dict['Vegetables'] = pd.DataFrame(columns=df.columns)
        food_dict['Vegetables'] = pd.concat([food_dict['Vegetables'], df.loc[[i]]])
    elif i in breakfast:
        if 'Breakfast' not in food_dict:
            food_dict['Breakfast'] = pd.DataFrame(columns=df.columns)
        food_dict['Breakfast'] = pd.concat([food_dict['Breakfast'], df.loc[[i]]])
    elif i in frozen:
        if 'Frozen' not in food_dict:
            food_dict['Frozen'] = pd.DataFrame(columns=df.columns)
        food_dict['Frozen'] = pd.concat([food_dict['Frozen'], df.loc[[i]]])
    elif i in drinks:
        if 'Drinks' not in food_dict:
            food_dict['Drinks'] = pd.DataFrame(columns=df.columns)
        food_dict['Drinks'] = pd.concat([food_dict['Drinks'], df.loc[[i]]])
    elif i in desserts:
        if 'Desserts' not in food_dict:
            food_dict['Desserts'] = pd.DataFrame(columns=df.columns)
        food_dict['Desserts'] = pd.concat([food_dict['Desserts'], df.loc[[i]]])
    elif i in snacks:
        if 'Snacks' not in food_dict:
            food_dict['Snacks'] = pd.DataFrame(columns=df.columns)
        food_dict['Snacks'] = pd.concat([food_dict['Snacks'], df.loc[[i]]])
    elif i in dry:
        if 'Dry' not in food_dict:
            food_dict['Dry'] = pd.DataFrame(columns=df.columns)
        food_dict['Dry'] = pd.concat([food_dict['Dry'], df.loc[[i]]])
    elif i in dairy:
        if 'Dairy' not in food_dict:
            food_dict['Dairy'] = pd.DataFrame(columns=df.columns)
        food_dict['Dairy'] = pd.concat([food_dict['Dairy'], df.loc[[i]]])

# Now, the food_dict dictionary contains DataFrames for each type of food
# Access them like food_dict['Meats'], food_dict['Fruits'], etc.


In [20]:
food_dict

{'Meats':                    Calories  Serving  Protein  Carbs    Fat  Calcium     Price
 chicken               107.0     56.0    21.43   0.00   1.79      0.0  5.496714
 steaks                  0.0      0.0    11.90   4.34   9.12     17.0  4.614918
 burgers               252.0    151.0    17.22   0.00  19.87     20.0  4.323078
 Pork (bacon, ham)       0.0      0.0     6.26  10.40   8.27     21.0  5.611676,
 'Breakfast':                   Calories  Serving  Protein  Carbs    Fat  Calcium     Price
 yogurt                94.0    170.0     2.94  15.88   2.94    118.0  4.861736
 cereal               387.0     31.0     6.45  83.87   3.23    323.0  5.647689
 Eggs                 143.0     63.0    12.70   0.00   9.52     63.0  6.031000
 Oatmeal              378.0     45.0    11.11  68.89   6.67     44.0  4.690788
 Pancake mix          333.0     36.0     5.56  75.00   0.00      0.0  5.331263
 cream cheese         321.0     28.0     7.14   7.14  32.14     71.0  5.361636
 Toaster pastries     37

In [21]:
meats_df = food_dict["Meats"]
fruits_df = food_dict["Fruits"] 
vegetables_df = food_dict["Vegetables"]
breakfast_df = food_dict['Breakfast'] 
frozen_df = food_dict["Frozen"]
drinks_df = food_dict["Drinks"]
desserts_df = food_dict["Desserts"]
snacks_df = food_dict["Snacks"]
dry_df = food_dict["Dry"]   
dairy_df = food_dict["Dairy"]

In [22]:
dairy_df

,Calories,Serving,Protein,Carbs,Fat,Calcium,Price
Milk,55.0,236.0,3.39,5.08,2.12,127.0,3.236960
Cheese,367.0,30.0,16.67,3.33,33.33,503.0,5.324084


In [23]:
# Item = m.addSet('Item',records=df_limited.index)
# p = m.addParameter('price',[Item],records=df_limited[["Price"]].to_numpy())
# cal = m.addParameter('calories',[Item],records=df_limited[["Calories"]].to_numpy())
# pro = m.addParameter('protein',[Item],records=df_limited[["Protein"]].to_numpy())
# carbs = m.addParameter('carbs',[Item],records=df_limited[["Carbs"]].to_numpy())
# fat = m.addParameter('fat',[Item],records=df_limited[["Fat"]].to_numpy())
# calcium = m.addParameter('calcium',[Item],records=df_limited[["Calcium"]].to_numpy())
# display(calcium.records)

In [24]:
Item = m.addSet('Item',records=df.index)
me = m.addSet("meat",domain = [Item],records=meats_df.index)
fruit = m.addSet("fruit",domain = [Item],records=fruits_df.index)
veg = m.addSet("vegetable",domain = [Item],records=vegetables_df.index)
bf = m.addSet("breakfast",domain = [Item],records=breakfast_df.index)
fr = m.addSet("frozen",domain = [Item],records=frozen_df.index)
dr = m.addSet("drink",domain = [Item],records=drinks_df.index)
de = m.addSet("dessert",domain = [Item],records=desserts_df.index)
dry = m.addSet("dry",domain = [Item],records=dry_df.index)
dairy = m.addSet("dairy",domain = [Item],records=dairy_df.index)
p = m.addParameter('price',[Item],records=df[["Price"]].to_numpy())
cal = m.addParameter('calories',[Item],records=df[["Calories"]].to_numpy())
pro = m.addParameter('protein',[Item],records=df[["Protein"]].to_numpy())
carbs = m.addParameter('carbs',[Item],records=df[["Carbs"]].to_numpy())
fat = m.addParameter('fat',[Item],records=df[["Fat"]].to_numpy())
calcium = m.addParameter('calcium',[Item],records=df[["Calcium"]].to_numpy())
display(bf.records)

,Item,element_text
0,yogurt,
1,cereal,
2,Eggs,
3,Oatmeal,
4,Pancake mix,
5,cream cheese,
6,Toaster pastries,
7,Frozen waffles,


In [25]:
cost = m.addVariable('cost','free',description="total cost")
x = m.addVariable('x','integer',domain = Item,description="number of servings for each item")


In [34]:
%%gams
scalar
    requirements /3000/;

alias(Item, i);
    
equations
    obj "minimize total meal cost"
    cal_constraint_up
    cal_constraint_low
    protein_constraint_up
    protein_constraint_low
    carb_constraint_up
    carb_constraint_low
    fat_constraint_up
    fat_constraint_low
    calcium_constraint_up
    calcium_constraint_low
    cost_constraint
    meat_constraint_up
    meat_constraint_low
    fruit_constraint_up
    fruit_constraint_low
    veg_constraint_up
    veg_constraint_low
    breakfast_constraint_up
    breakfast_constraint_low
    frozen_constraint_up
    frozen_constraint_low
    drink_constraint_up
    drink_constraint_low
    dessert_constraint_up
    dessert_constraint_low
    dry_constraint_up
    dry_constraint_low
    dairy_constraint_up
    dairy_constraint_low;
    
obj..
    cost =e= sum(Item, price(Item)*x(Item));

# Do not go over budget
cost_constraint..
    cost =l= 100;
    
# Make sure we meet our nutritional goals
cal_constraint_up..
    sum(Item, x(Item) * calories(Item)) =l= 3500;
    
cal_constraint_low..
    sum(Item, x(Item) * calories(Item)) =g= 3000;
    
protein_constraint_up..
    sum(i, x(i) * protein(i)) =l= 73;
    
protein_constraint_low..
    sum(i, x(i) * protein(i)) =g= 45;
    
carb_constraint_up..
    sum(i, x(i) * carbs(i)) =l= 503;
    
carb_constraint_low..
    sum(i, x(i) * carbs(i)) =g= 308;
    
fat_constraint_up..
    sum(i, x(i) * fat(i)) =l= 120;
    
fat_constraint_low..
    sum(i, x(i) * fat(i)) =g= 69;
    
calcium_constraint_up..
    sum(i, x(i) * calcium(i)) =l= 200;
    
calcium_constraint_low..
    sum(i, x(i) * calcium(i)) =g= 100;
    
    
# Make sure we diversify the food groups we are eating     
meat_constraint_up..
    sum(meat, calories(meat) * x(meat)) =l= 1000;
    
meat_constraint_low..
    sum(meat, calories(meat) * x(meat)) =g= 100;
    
fruit_constraint_up..
    sum(fruit, calories(fruit) * x(fruit)) =l= 1000;
    
fruit_constraint_low..
    sum(fruit, calories(fruit) * x(fruit)) =g= 100;
    
veg_constraint_up..
    sum(vegetable, calories(vegetable) * x(vegetable)) =l= 1000;
    
veg_constraint_low..
    sum(vegetable, calories(vegetable) * x(vegetable)) =g= 100;
    
breakfast_constraint_up..
    sum(breakfast, calories(breakfast) * x(breakfast)) =l= 500;
    
breakfast_constraint_low..
    sum(breakfast, calories(breakfast) * x(breakfast)) =g= 50;
    
frozen_constraint_up..
    sum(frozen, calories(frozen) * x(frozen)) =l= 100;
    
frozen_constraint_low..
    sum(frozen, calories(frozen) * x(frozen)) =g= 0;
    
drink_constraint_up..
    sum(drink, calories(drink) * x(drink)) =l= 100;
    
drink_constraint_low..
    sum(drink, calories(drink) * x(drink)) =g= 0;
    
dessert_constraint_up..
    sum(dessert, calories(dessert) * x(dessert)) =l= 50;
    
dessert_constraint_low..
    sum(dessert, calories(dessert) * x(dessert)) =g= 0;
    
dry_constraint_up..
    sum(dry, calories(dry) * x(dry)) =l= 250;
    
dry_constraint_low..
    sum(dry, calories(dry) * x(dry)) =g= 20;
    
dairy_constraint_up..
    sum(dairy, calories(dairy) * x(dairy)) =l= 150;
    
dairy_constraint_low..
    sum(dairy, calories(dairy) * x(dairy)) =g= 0;
    
# X cannot be negative
x.lo(i) = 0;
 
model plan /obj 
    cal_constraint_up
    cal_constraint_low
    protein_constraint_up
    protein_constraint_low
    carb_constraint_up
    carb_constraint_low
    fat_constraint_up
    fat_constraint_low
    calcium_constraint_up
    calcium_constraint_low
    cost_constraint
    meat_constraint_up
    meat_constraint_low
    fruit_constraint_up
    fruit_constraint_low
    veg_constraint_up
    veg_constraint_low
    breakfast_constraint_up
    breakfast_constraint_low
    frozen_constraint_up
    frozen_constraint_low
    drink_constraint_up
    drink_constraint_low
    dessert_constraint_up
    dessert_constraint_low
    dry_constraint_up
    dry_constraint_low
    dairy_constraint_up
    dairy_constraint_low
    /;
solve plan using mip minimizing cost;



,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),44.276,30,88,MIP,CPLEX,0.006


In [27]:
display(x.records)

None